<p style="text-align: center; color: #D6A4FF; font-family: 'Montserrat'; font-weight: bold; font-size: 36px">Notebook de exemplo para como corretamente fazer o ETL dos dados</p>

<ul style="font-family: 'Montserrat'; color: white; background-color: #FFFFF; font-size: 22px; padding: 5px; border-radius: 5px;">
  <li>Apresenta apenas como organizar os dados com um exemplo em JSON.</li>
  <li>Quais devem ser os atributos para envio no Postgre DB.</li>
</ul>

In [4]:
import pandas as pd
import numpy as np
import os, json
import itertools

<p style="text-align: center; color: #D6A4FF; font-family: 'Montserrat'; font-weight: bold; font-size: 22px;">Exemplo Gmail API</p>

<ul style="font-family: 'Montserrat'; color: white; background-color: #FFFFF; font-size: 16px; padding: 5px; border-radius: 5px;">
  <li>Como coletar os dados das interações via email dos colaboradores</li>
  <li>Como juntar esses dados com o que foi coletado do Calendar API anteriormente</li>
</ul>

In [5]:
# Importando os dados de exemplo do GMAIL API
# Obviamente esses dados serão extraídos diretamente da chamada da GMAIL API, aqui é apenas um exemplo

with open('gmail_data_example.json', 'r') as file:
    # Como anteriormente, "coletando" os dados de maneira manual
    data = json.load(file)
    result_gmail = []
    # Iterando sobre cada email, aqui nós criamos as interações entre todas as pessoas que participaram do email, independente se foi o indivíduo que enviou o email
    for email in data['messages']:
        # Data do envio do email sendo coletado através do atributo messages
        data_email = [conteudo['value'] for conteudo in email['payload']['headers'] if conteudo['name'] == 'Date'][0]

        # Aqui coletamos os emails, tem casos em que os colaboradores em cópia são mais de um, como Cc: "user@, user2@" veja como não é uma lista, por isso utilizamos o split
        lista_emails = set([
            email.strip()
            for conteudo in email['payload']['headers'] if conteudo['name'] != 'Date'
            for email in conteudo['value'].split(',')
        ])

        # Criando todas as combinações possíveis não repetidas dos emails citados no envio
        combinations = list(itertools.combinations(lista_emails, 2))
        result_gmail.extend([
            {
                'id': email['id'],
                'sender':dupla[0],
                'receiver':dupla[1],
                'data': pd.Timestamp(data_email).date(),
                'de_onde':'gmail_api'
            }
            for dupla in combinations
        ])

result_gmail = pd.DataFrame(result_gmail) ; result_gmail

,id,sender,receiver,data,de_onde
0,message1,user2@company.com,user1@company.com,2025-03-01,gmail_api
1,message1,user2@company.com,user4@company.com,2025-03-01,gmail_api
2,message1,user2@company.com,user5@company.com,2025-03-01,gmail_api
3,message1,user2@company.com,user3@company.com,2025-03-01,gmail_api
4,message1,user1@company.com,user4@company.com,2025-03-01,gmail_api
5,message1,user1@company.com,user5@company.com,2025-03-01,gmail_api
6,message1,user1@company.com,user3@company.com,2025-03-01,gmail_api
7,message1,user4@company.com,user5@company.com,2025-03-01,gmail_api
8,message1,user4@company.com,user3@company.com,2025-03-01,gmail_api
9,message1,user5@company.com,user3@company.com,2025-03-01,gmail_api


<p style="text-align: center; color: #D6A4FF; font-family: 'Montserrat'; font-weight: bold; font-size: 22px;">Caso de agrupamento</p>

<ul style="font-family: 'Montserrat'; color: white; background-color: #FFFFF; font-size: 16px; padding: 5px; border-radius: 5px;">
  <li>Perceba como acima, temos casos de ID iguais com senders e receivers iguais, precisamos resolver esse problema!</li>
  <li>Isso acontece pois extraímos os dados de duas caixas de email de users diferentes, trazendo a mesma reunião e por isso precisamos eliminar essa duplicação para isso criamos um hash_id que mapeia a relação</li>
</ul>

In [6]:
# Utilize dessa função para criar o hashmap dos colaboradores da organização
# Queremos mapear a relação entre 2 colaboradores independente da ordem das colunas
import hashlib
def generate_hash(sender, receiver):
    interaction = f"{min(sender, receiver)}-{max(sender, receiver)}"
    return hashlib.sha256(interaction.encode()).hexdigest()

result_gmail['hash_id'] = result_gmail.apply(lambda row: generate_hash(row['sender'], row['receiver']), axis=1)

In [9]:
result_gmail[result_gmail['id'] == 'message2']['hash_id'].value_counts()

812490b0a51ec92f66fb56e6dadabf5506ce933611b5081d6a9ffadb7b71ffda    2
1cbfc5abcc325a852e83c053c2e07fbe51e72a0f714fbe0c636e143108bdc424    2
aedf17d1a4886adc4ac119c3c24b60bf4a094860cfa92d2e6110ee6096512430    2
3ba2a91bb1cc1c312abcd0254797d2481c348a5e0f94bc99c12ebdb96a7dd271    2
ff7cec764662b40b3046565d2e5cf70541ac01ecafcd6ad0a5392eff1bd926b7    2
bf43dff89718ba80dcf79c25eab38cfde2733120408452a0f0f6c7b933f50013    2
9c503f98cceafa28bcc895b97d9d4334cc2a79828a771da2fc3414c17ecc61f2    2
95f160e191e39cda40f6b3e3c1e30773911fec1d9f1a8c48589275b96582545c    2
fc5d72c791baee47fdaaf5bdcf1e72f8aa81225e0cca3ea40e07d53123265259    2
68d462646e2ce1820a9ec32974418fa46e4efa50034b450cbbd52dff38dccd24    2
Name: hash_id, dtype: int64

<p style="text-align: center; color: #D6A4FF; font-family: 'Montserrat'; font-weight: bold; font-size: 22px;">Caso de agrupamento</p>

<ul style="font-family: 'Montserrat'; color: white; background-color: #FFFFF; font-size: 16px; padding: 5px; border-radius: 5px;">
  <li>Perceba como acima, como realmente para um ID temos 2 hash_ids iguais, precisamos agrupar e eliminar duplicatas</li>
</ul>

In [ ]:
result_gmail.groupby(['id','hash_id']).agg(
    sender=('sender','max'),
    receiver=('receiver','max'),
    data=('data','max'),
    de_onde=('de_onde','max')
).reset_index()

<p style="text-align: center; color: #D6A4FF; font-family: 'Montserrat'; font-weight: bold; font-size: 22px;">Envio dos dados para o Postgre DB</p>

<ul style="font-family: 'Montserrat'; color: white; background-color: #FFFFF; font-size: 16px; padding: 5px; border-radius: 5px;">
  <li>Agora com os dataframes de Calendar e Mail prontos, basta fazer um pd.concat(,axis=0) deles e realizar o envio ao Postgre DB na Cloud com as credenciais fornecidas.</li>
</ul>

### Critérios de aceite:

* É possível existir id duplicados.
* É possível existir hash_id duplicados.
* Não se pode possuir id e hash_id duplicados.
* É necessário realizar as combinações par a par de todos os participantes da reunião utilizando o itertools como foi demonstrado.